In [ ]:
# 동아디지털아카이브(1996년 10월 14일 이전 기사, 지면 기사) 목록, 본문
# 1996년 10월 14일 이후 기사는 홈페이지와 디지털아카이브 두 군데에 아카이빙되어 있으므로, 이 코드는 1996년 10월 14일 이전에만 사용할 것.

import time
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# 크롬 드라이버 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 필요시 활성화
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화
chrome_options.add_argument("--disable-gpu")  # GPU 가속 비활성화

# 웹 드라이버 초기화
driver = webdriver.Chrome(options=chrome_options)

# 사용자 정보
user_id = 'id'
user_pw = 'pw'

# 검색어
keywords = [
    "검색어",
]
# 검색 기간 설정
start_date = "20240601"
end_date = "20240831"

collected_urls = set()

# 로그인 페이지로 이동
driver.get('https://secure.donga.com/membership/login.php?gourl=https%3A%2F%2Fwww.donga.com')

# ID와 PW 입력 및 로그인
driver.find_element(By.NAME, 'email').send_keys(user_id)
driver.find_element(By.NAME, 'bpw').send_keys(user_pw)
driver.find_element(By.ID, 'loginbtn').click()

# 로그인 후 대기
time.sleep(1)

# 뉴스 라이브러리 페이지로 이동 및 검색어 리스트 설정
base_url = "https://www.donga.com/archive/newslibrary"


# 각 검색어에 대해 기사 검색 및 정보 수집
articles = []

def parse_articles():
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")
    articles = soup.find_all("div", class_="tit_cont")
    return articles


def extract_article_info(article):
    url_tag = article.find("a")
    if not url_tag:
        return None

    title_tag = url_tag.find("span", class_="titie")
    date_tag = url_tag.find("span", class_="date")
    position_tag = url_tag.find("span", class_="position")

    if not title_tag or not date_tag or not position_tag:
        return None

    title = title_tag.text.strip()
    url = url_tag["href"]
    date = date_tag.text.strip()
    position = position_tag.text.strip()

    position_parts = position.split(" ")
    page = position_parts[1] if len(position_parts) >= 2 else position
    section = " ".join(position_parts[2:]) if len(position_parts) >= 2 else ""

        # 중복 URL 체크: 이미 수집된 URL이면 None 반환
    if url in collected_urls:
        return None

    # 새로운 URL이면 집합에 추가
    collected_urls.add(url)

    return (title, url, date, page, section)

for keyword in keywords:
    page_number = 1

    while True:
        try:
            print(f"{keyword} - {page_number} 건을 처리 중입니다...")
            driver.get(f"{base_url}?p={page_number}&query={keyword}&range=1&search_date=&sdate={start_date}&edate={end_date}&sec_top=&sorting=2&option=1&l=20")
            time.sleep(2)

            articles = parse_articles()
            if not articles:
                break
            
            for index, article in enumerate(articles):
                info = extract_article_info(article)
                if info:
                    title, url, date, page, section = info

                    # 링크 클릭하여 기사 페이지로 이동
                    driver.get(url)
                    time.sleep(2)  # 페이지가 로드될 때까지 대기
                    article_page = driver.page_source
                    article_soup = BeautifulSoup(article_page, "html.parser")

                    # 원문, 독음 추출 및 입력
                    original_content_tag = article_soup.find("div", id="article_origin")
                    transfer_content_tag = article_soup.find("div", id="article_transfer")
                    ori_content = original_content_tag.find("div", class_="article_txt").get_text(separator="\n").strip().replace('\n', ' ') if original_content_tag else ""
                    trans_content = transfer_content_tag.find("div", class_="article_txt").get_text(separator="\n").strip().replace('\n', ' ') if transfer_content_tag else ""
                    if trans_content.startswith('='):
                       trans_content = "'" + trans_content

                    # 글쓴이 추출
                    writer_tag = transfer_content_tag.find("span", class_="reporter")

                    # 글쓴이 입력
                    writer = writer_tag.text.strip() if writer_tag else ""
                    
                    articles.append({
                        "media": "동아일보",
                        "title": title,
                        "date": date,
                        "section": section,
                        "article_type": '',
                        'writer': writer,
                        "page": page,
                        "url": url,
                        'content': trans_content
                        })

        except Exception as e:
            print(f"{page_number} 페이지 처리 중 오류 발생: {e}")

        # 다음 페이지로 이동
        page_number += 20
        next_page_url = f"{base_url}?p={page_number}&query={keyword}&range=1&search_date=&sdate=&edate=&sec_top=&sorting=2&option=1&l=20"
        driver.get(next_page_url)
        time.sleep(2)

        # 더 이상 페이지가 없으면 루프 종료
        if "현재 페이지의 기사목록이 없습니다." in driver.page_source:
            print("더 이상 다음 페이지가 없습니다.")
            break

# 데이터프레임 생성 및 중복 기사 제거
df = pd.DataFrame(articles)
df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df = df.drop_duplicates(subset=['url'], keep='first')

# 엑셀 파일 저장 경로 설정
output_file = os.path.join(os.getcwd(), f'1동아일보 {start_date}~{end_date}.xlsx')

# 데이터프레임 엑셀 저장
df.to_excel(output_file, index=False)

print("파일 저장 완료!")


In [ ]:
# 홈페이지 뉴스(1996년 10월 14일 이후 기사) 목록

import time
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# 크롬 드라이버 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 필요시 활성화
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화
chrome_options.add_argument("--disable-gpu")  # GPU 가속 비활성화

# 웹 드라이버 초기화
driver = webdriver.Chrome(options=chrome_options)

# 검색어
keywords = [
    "검색어",
]

# 검색 기간 설정
start_date = "20000101"
end_date = "20240531"
collected_urls = set()
articles = []  # 모든 기사 정보를 저장할 리스트

# 각 키워드에 대해 기사 검색 및 정보 수집
for keyword in keywords:
    page_number = 1

    while True:
        try:
            print(f"{keyword} 기사 {page_number} 건를 처리 중입니다...")
            driver.get(f"https://www.donga.com/news/search?p={page_number}&query={keyword}&sorting=1&check_news=91&sorting=1&search_date=5&v1={start_date}&v2={end_date}&more=1")
            time.sleep(2)

            # 페이지 소스 파싱
            soup = BeautifulSoup(driver.page_source, "html.parser")
            li_elements = soup.find("ul", class_="row_list").find_all("li", class_="") if soup.find("ul", class_="row_list") else []

            if not li_elements:  # li 태그가 없는 경우 루프 종료
                break  

            for li in li_elements:
                article = li.find("div", class_="news_body")
                if article:
                    title_tag = article.find("h4", class_="tit")
                    url_tag = title_tag.find("a") if title_tag else None
                    date_tag = article.find("span", class_="date")

                    if not title_tag or not url_tag or not date_tag:
                        continue

                    title = title_tag.get_text(strip=True)
                    url = url_tag["href"]
                    date = date_tag.get_text(strip=True)

                    # 중복 URL 체크
                    if url in collected_urls:
                        continue
                    collected_urls.add(url)

                    # 기사 정보를 articles 리스트에 추가
                    articles.append({
                        "media": "동아일보",
                        "title": title,
                        "date": date,
                        "section": '', 
                        "article_type": '',
                        'writer': '',  
                        "page": '',  
                        "url": url,
                        'content': ''  
                    })

            # 유효한 기사가 10개 미만이면 루프 종료
            if len(li_elements) < 10:
                break

        except Exception as e:
            print(f"{page_number} 페이지 처리 중 오류 발생: {e}")

        page_number += 10

# 데이터프레임 생성 및 중복 기사 제거
df = pd.DataFrame(articles)
df = df.drop_duplicates(subset=['url'], keep='first')

# 엑셀 파일 저장 경로 설정
output_file = os.path.join(os.getcwd(), f'1동아일보 {start_date}~{end_date}.xlsx')

# 데이터프레임 엑셀 저장
df.to_excel(output_file, index=False)

print("파일 저장 완료!")


In [ ]:
# 본문 추출

import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# 1. 출처와 파일명 변수 설정
file_name = '파일명'  # 파일명 설정
base_path = rf'경로'  # 경로 설정

# 2. 임시 파일 경로와 최종 파일 경로 생성
temp_output_path = os.path.join(base_path, f'{file_name}.xlsx')
# 임시 파일 에러를 대비해 본 파일에 덮어쓴다.
output_path = os.path.join(base_path, f'{file_name}_본문.xlsx')  # 최종 파일 경로
error_log_path = os.path.join(base_path, 'error_log.txt')  # 오류 로그 경로

# 3. 파일 경로 생성 및 엑셀 파일 불러오기
input_file = os.path.join(base_path, f'{file_name}.xlsx')
df = pd.read_excel(input_file, engine='openpyxl')

# Selenium 헤드리스 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# ChromeDriver 설정 없이 크롬 브라우저 실행
driver = webdriver.Chrome(options=chrome_options)

# URL에서 본문과 서브 타이틀을 스크랩하는 함수 정의
def get_content_and_subtitle_from_url(url):
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 서브 타이틀 추출 (h2.sub_tit)
        subtitle_tag = soup.find('h2', class_='sub_tit')
        subtitle = ""
        if subtitle_tag:
            subtitle = subtitle_tag.get_text(separator='\n', strip=True)
            subtitle_tag.decompose()  # 부제를 본문에서 제거

        # 본문 추출
        article_body = None
        if url.startswith('https://www.donga.com/news/'):
            article_body = soup.find('section', class_='news_view')
        else:
            article_body = soup.find('div', class_='article_txt')

        if article_body:
            # 제거할 태그와 클래스 리스트
            tags_to_remove = [
                ('figure', 'img_cont'),  
                ('figcaption', None)    
            ]

            # 태그를 반복문으로 순회하면서 제거
            for tag, class_name in tags_to_remove:
                if class_name:
                    elements = article_body.find_all(tag, class_=class_name)
                else:
                    elements = article_body.find_all(tag)
                
                for element in elements:
                    element.decompose()

            # 본문 텍스트 추출
            content = article_body.get_text(separator='\n', strip=True)
        else:
            content = ""

        return content, subtitle
    except Exception as e:
        print(f"Error fetching URL {url}: {e}")
        return None, None

# 진행 중인 기사 스크래핑 상태를 트래킹할 리스트와 로그 파일
error_log = []
total_articles = len(df)

# 각 행의 URL에서 본문과 서브 타이틀 내용을 스크랩하여 content와 sub-title 컬럼에 저장합니다.
for index, row in df.iterrows():
    print(f"Processing {index + 1}/{total_articles}: {row['url']}")
    content, subtitle = get_content_and_subtitle_from_url(row['url'])

    if content is not None and content.startswith('='):
        content = "'" + content

    df.at[index, 'content'] = content if content is not None else ""
    df.at[index, 'sub-title'] = subtitle  # 서브 타이틀이 없으면 빈 문자열로 저장

    # 100개마다 파일 저장
    if (index + 1) % 100 == 0:
        df.to_excel(temp_output_path, index=False)
        print(f"임시 파일을 저장했습니다: {temp_output_path}")

# 웹드라이버 종료
driver.quit()

# 변경된 데이터를 파일로 저장합니다.
df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df = df[~df['content'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df.to_excel(output_path, index=False)

print("본문 내용을 성공적으로 스크랩하여 저장했습니다.")

# 에러 로그 파일 저장
if error_log:
    with open(error_log_path, 'w') as f:
        for url in error_log:
            f.write(f"{url}\n")
    print(f"에러 로그가 저장되었습니다: {error_log_path}")


      
